# Functional API

## import packages

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## import keras packages

In [2]:
%env KERAS_BACKEND=tensorflow

# Keras functions
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input
from keras.layers import concatenate, add
from keras.optimizers import SGD, Adam

# Keras dataset
from keras.datasets import mnist

# Keras utils
from keras.utils import np_utils

env: KERAS_BACKEND=tensorflow


Using TensorFlow backend.


## import dataset MNIST

In [3]:
(x_train0, y_train0), (x_test0, y_test0) = mnist.load_data()

In [4]:
# 拉直
x_train = x_train0.reshape(60000, 784)
x_test = x_test0.reshape(10000, 784)
# scaling
x_test -= x_train.min()
x_train -= x_train.min()
x_test = x_test/x_train.max()
x_train = x_train/x_train.max()
# 1-hot-encoding
y_train = np_utils.to_categorical(y_train0, 10)
y_test = np_utils.to_categorical(y_test0, 10)

## Functional API

### draw & construct my model

My graph in mermaid code(https://hackmd.io/H9F_DIr2Sgqx8IevwuCWQQ?both).
```mermaid
graph LR
input[<br/><br/><br/><br/>784<br/><br/><br/><br/>] --> L11[<br/><br/>40<br/><br/>]
input --> L12[<br/><br/>40<br/><br/>]

L11 --> L21[<br/>20<br/>]
L11 --> L22[<br/>20<br/>]
L12 --> L22
L12 --> L23[<br/>20<br/>]

L21 --> L31[10]
L21 --> L32[10]
L22 --> L32
L22 --> L33[10]
L23 --> L33
L23 --> L34[10]

L31 --> L41[10]
L32 --> L41
L32 --> L42[10]
L33 --> L42
L33 --> L43[10]
L34 --> L43


L41 --> L51[10]
L42 --> L51 
L42 --> L52[10]
L43 --> L52

L51 --> output[10]
L52 --> output
```

In [5]:
# input layer
_input = Input(shape = (784, ))

# 1st layer
L11 = Dense(40, activation = 'relu')(_input)
L12 = Dense(40, activation = 'relu')(_input)

# 2nd layer
L21 = Dense(20, activation = 'relu')(L11)
L22 = Dense(20, activation = 'relu')(concatenate([L11, L12]))
L23 = Dense(20, activation = 'relu')(L12)

# 3rd layer
L31 = Dense(10, activation = 'relu')(L21)
L32 = Dense(10, activation = 'relu')(concatenate([L21, L22]))
L33 = Dense(10, activation = 'relu')(concatenate([L22, L23]))
L34 = Dense(10, activation = 'relu')(L23)

# 4th layer
L41 = Dense(10, activation = 'relu')(concatenate([L31, L32]))
L42 = Dense(10, activation = 'relu')(concatenate([L32, L33]))
L43 = Dense(10, activation = 'relu')(concatenate([L33, L34]))

# 5th layer
L51 = Dense(10, activation = 'relu')(concatenate([L41, L42]))
L52 = Dense(10, activation = 'relu')(concatenate([L42, L43]))

# output layer
_output = Dense(10, activation = 'softmax')(concatenate([L51, L52]))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
model = Model(_input, _output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 40)           31400       input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 40)           31400       input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 80)           0           dense_1[0][0]                    
                                                                 dense_2[0][0]                    
__________

In [7]:
model.compile(optimizer = Adam(lr = 0.0009487),
              loss = "categorical_crossentropy",
              metrics = ['acc'])

### fit the model

In [8]:
model.fit(x_train, y_train,
          batch_size = 100,
          epochs = 20, 
          validation_data = (x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 4s 73us/step - loss: 0.4525 - acc: 0.8636 - val_loss: 0.1803 - val_acc: 0.9467
Epoch 2/20
60000/60000 [==============================] - 3s 43us/step - loss: 0.1559 - acc: 0.9547 - val_loss: 0.1420 - val_acc: 0.9567
Epoch 3/20
60000/60000 [==============================] - 3s 43us/step - loss: 0.1150 - acc: 0.9654 - val_loss: 0.1334 - val_acc: 0.9614
Epoch 4/20
60000/60000 [==============================] - 3s 43us/step - loss: 0.0969 - acc: 0.9713 - val_loss: 0.1090 - val_acc: 0.9678
Epoch 5/20
60000/60000 [==============================] - 3s 43us/step - loss: 0.0782 - acc: 0.9768 - val_loss: 0.1156 - val_acc: 0.9670
Epoch 6/20
60000/60000 [==============================] - 3s 43us/step - loss: 0.0662 - acc: 0.9802 - val_loss: 0.1009 - val_acc: 0.9718
Epoch 7/20
60000/60000 [==============================] - 3s 43us/step - loss: 0

### summary

Learning rate 好好選才不會爆開QQ